In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import os
from oiffile import imread
import random
import skimage
from skimage.transform import resize
import matplotlib
import glob
import torch
import sys
sys.path.insert(0, '..')
import scipy.io as sio
import matplotlib.patches as patches
from skimage.measure import label,regionprops
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.transform import rescale
from sklearn.metrics import roc_curve, auc
import os, os.path
from skimage.draw import circle

In [2]:
# binary masks
def make_labels(img,ys,xs,radius=4):
    labels = np.zeros(img.shape[1:])
    for xv,yv in zip(xs,ys):
        rr,cc = circle(xv,yv,radius,labels.shape)
        
        labels[rr,cc]=1
    return labels

# training images
def make_training_set(labels,indexes):
    train_images = []
    train_labels = []
    for i in indexes:
        if len(labels[i])==0:
            d = np.zeros_like(collman[0,0,:,:])
        else:
            d = make_labels(collman[:,0],np.array(labels[i])[:,0],np.array(labels[i])[:,1])
        train_images.append(collman[:,i])
        train_labels.append(d)
    return train_images,train_labels

In [3]:
# image number
i = 1
# generate image name
if i<10:
    n = '000%s' % (i)
elif i <100:
    n = '00%s' % (i)
else:
    n = '0%s' % (i)

fdir = r'E:\pcp2cre_syptom_568_mglur1_1to200_647_1to250_vgat_1to200_488_1to250\FV10__20190507_224650_flocculusA/Track%s' % (n)
# open image
im_name = 'Image%s_01.oib' % (n)
image = imread(os.path.join(fdir,im_name))
file_name= 'Track%s' % (n)

In [4]:
# randomly select z-slice to annnotate
z_slice=random.randint(0,15)
print(z_slice)

if z_slice < 10:
    v1 = str(z_slice)
    v = '0%s' % (v1)
else:
    v = str(z_slice)

# open a txt file for logging annotation
annotation_filename = "%s_%s\n" % (file_name,v)
with open("../../datasets/flocculusA/imagesUsed-flocculusA.txt", "a") as fin:
    fin.write(annotation_filename)

# load and clip pre-computed medians of slices
medianTotal= np.double(np.load("../../datasets/flocculusA/normalizationMatrices/medianTotal.npy"))
medianclipped0 = np.maximum(medianTotal[0],np.max(medianTotal[0])/3)
medianclipped1 = np.maximum(medianTotal[1],np.max(medianTotal[1])/3)
medianclipped2 = np.maximum(medianTotal[2],np.max(medianTotal[2])/3)

# make two copies of image for the selected slice
image_save = np.double(image[:,z_slice,:,:].transpose(1,2,0))
image_view = np.double(image[:,z_slice,:,:].transpose(1,2,0))
# normalize the image used for view
image_view[:,:,0] = image_view[:,:,0]/medianTotal[0][z_slice]
image_view[:,:,1] = image_view[:,:,1]/medianTotal[1][z_slice]
image_view[:,:,2] = 0
# normalize the image used for saving
image_save[:,:,0] = image_save[:,:,0]/medianclipped0[z_slice]
image_save[:,:,1] = image_save[:,:,1]/medianclipped1[z_slice]
image_save[:,:,2] = image_save[:,:,2]/medianclipped2[z_slice]
pt=[]
# manually anotate the selected image
%matplotlib inline
%matplotlib qt
plt.figure(figsize=(30,30))
plt.imshow(image_view[:,:,1])
plt.show()
pt = np.asarray(plt.ginput(150,timeout = -1))

# save raw coordinates
rawStr="../../datasets/flocculusA/rawCoordinates/%s_%s_rawCoords.npy" % (file_name,v)
np.savetxt(rawStr, pt)
# resize and cut image into 4 smaller images with overlapping segments 
image_save = resize(image_save,(400,400), order=1, preserve_range=True)
h=400
w=400
tol=30

UL=(image_save[0:int(h/2+tol),0:int(w/2+tol),:])       
UR=(image_save[0:int(h/2+tol),int(w/2-tol):w,:])
LL=(image_save[int(h/2-tol):h,0:int(w/2+tol),:])
LR=(image_save[int(h/2-tol):h,int(w/2-tol):w,:])   
smallPictures=[UL,UR,LL,LR]

# sort coordinates of four smaller images
b = [[] for i in range(0,4)]    
for j in pt:
    if j[0] < 460 and j[1] < 460:
        point = np.copy(j)
        (b[0]).append(.5*np.array(point))

    if j[0] > 340 and j[1] < 460:
        point = np.copy(j)
        point[0]=point[0]-340
        (b[1]).append(.5*np.array(point))
      
    if j[0] < 460 and j[1] > 340:
        point = np.copy(j)
        point[1]=point[1]-340
        (b[2]).append(.5*np.array(point))
        
    if j[0] > 340 and j[1] > 340:
        point = np.copy(j)
        point[0]=point[0]-340
        point[1]=point[1]-340
        (b[3]).append(.5*np.array(point))

collman = np.transpose((np.stack(smallPictures,axis=3)),(2,3,0,1))
train_images,train_labels = make_training_set(b,range(0,4))
smallPicturesNames=['UL','UR','LL','LR']
# save individual training images, training masks, and coordinates
for j in range(0,4): 
    coordsStr="../../datasets/flocculusA/trainingCoordinates/%s_%s_%s_coords" % (file_name,v,smallPicturesNames[j])
    maskStr="../../datasets/flocculusA/trainingMasks/%s_%s_%s_mask.npy" % (file_name,v,smallPicturesNames[j])
    imageStr="../../datasets/flocculusA/trainingImages/%s_%s_%s_image.npy" % (file_name,v,smallPicturesNames[j])
    np.save(maskStr, train_labels[j])
    np.save(imageStr, train_images[j])
    np.savetxt(coordsStr, b[j], delimiter=',')

4
